In [65]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [66]:
csv_file_path = '/home/lucas/UFOP/ple_2020/analise_midias_sociais/final-work/data/all_pp_posts_bolsonaro.csv'
df_candidate = pd.read_csv(csv_file_path,encoding = 'utf-8')

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [62]:
df_candidate.head(15)

,created_time,id,m_m_p_replaced_to_abbreviation,m_m_p_replaced_to_empty,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
0,2020-03-22T18:47:55+0000,211857482296579_1826486634166981,agora videoconferencia presidente jb president...,agora videoconferencia presidente presidente...,23151,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,129465,147,318,21,112,10345,True
1,2020-03-22T16:30:39+0000,211857482296579_1826339070848404,inscricoes selecao profissionais atuacao progr...,inscricoes selecao profissionais atuacao progr...,5703,mobile_status_update,None,52238,128,61,5,43,2514,True
2,2020-03-22T13:37:26+0000,211857482296579_1826130007535977,ministerio saude ministro mandetta reuniao vir...,ministerio saude ministro mandetta reuniao vir...,11818,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,97986,73,141,16,65,6292,True
3,2020-03-22T11:07:01+0000,211857482296579_1825937284221916,governo federal distribui milhoes testes rapid...,governo federal distribui milhoes testes rapid...,39284,mobile_status_update,None,165539,88,393,20,68,16747,True
4,2020-03-22T02:06:48+0000,211857482296579_1825422034273441,chama precaucao medicamento barato caso venha ...,chama precaucao medicamento barato caso venha ...,27236,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9052732...,162513,968,318,88,509,20723,True
5,2020-03-21T22:22:48+0000,211857482296579_1825061764309468,reconheco seriedade momento temor muitos brasi...,reconheco seriedade momento temor muitos brasi...,40794,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9065012...,305358,918,478,266,1267,48834,True
6,2020-03-21T18:36:01+0000,211857482296579_1824728371009474,hospital albert einstein possivel cura pacient...,hospital albert einstein possivel cura pacient...,201803,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/75...,215237,480,815,60,431,34278,True
7,2020-03-21T11:33:30+0000,211857482296579_1824261207722857,dois dedos prosa ratinho assista link youtube,dois dedos prosa ratinho assista link youtube,33971,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/83...,106117,336,317,113,624,15368,True
8,2020-03-21T02:37:00+0000,211857482296579_1823872341095077,assinei medida provisoria deixando claro ser c...,assinei medida provisoria deixando claro ser c...,9174,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/s720x72...,112873,181,200,106,1121,7582,True
9,2020-03-21T00:16:41+0000,211857482296579_1823743497774628,via eduardo jb,via eduardo,8150,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p600x60...,79762,435,111,13,160,6480,True


In [48]:
n_total_posts = len(df_candidate)
print('número total de posts: {0}'.format(str(n_total_posts)))

número total de posts: 3646


In [49]:
n_posts_with_textual_messages = df_candidate[df_candidate.has_textual_message == True].shape[0]
n_posts_without_textual_messages = df_candidate[df_candidate.has_textual_message == False].shape[0]

print('número de posts com mensagens textuais: {0}'.format(str(n_posts_with_textual_messages)))
print('número de posts sem mensagens textuais: {0}'.format(str(n_posts_without_textual_messages)))

número de posts com mensagens textuais: 3394
número de posts sem mensagens textuais: 252


In [50]:
# Devemos remover esses posts sem menssagem antes de continuar a nossa análise?
# renomear depois o df caso sim

#filter_mask  = df_candidate['has_textual_message'] == True
#df_candidate = df_candidate[filter_mask]
#print('número de posts do df desconsiderando os posts sem menssagens textuais: {0}'.format(str(len(df_candidate))))

In [51]:
# separando o df de treinamento (30%) e o df de testes(70%) de maneira 'embaralhada'
df_training, df_test = train_test_split(df_candidate,train_size=0.7,test_size=0.3,shuffle=False)

In [40]:
print('número de posts do conjunto de treinamento (70%): {0}'.format(str(len(df_training))))
df_training.head(5)

número de posts do conjunto de treinamento (70%): 2552


,created_time,id,m_m_p_replaced_to_abbreviation,m_m_p_replaced_to_empty,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
0,2020-03-22T18:47:55+0000,211857482296579_1826486634166981,agora videoconferencia presidente jb president...,agora videoconferencia presidente presidente...,23151,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,129465,147,318,21,112,10345,True
1,2020-03-22T16:30:39+0000,211857482296579_1826339070848404,inscricoes selecao profissionais atuacao progr...,inscricoes selecao profissionais atuacao progr...,5703,mobile_status_update,None,52238,128,61,5,43,2514,True
2,2020-03-22T13:37:26+0000,211857482296579_1826130007535977,ministerio saude ministro mandetta reuniao vir...,ministerio saude ministro mandetta reuniao vir...,11818,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,97986,73,141,16,65,6292,True
3,2020-03-22T11:07:01+0000,211857482296579_1825937284221916,governo federal distribui milhoes testes rapid...,governo federal distribui milhoes testes rapid...,39284,mobile_status_update,None,165539,88,393,20,68,16747,True
4,2020-03-22T02:06:48+0000,211857482296579_1825422034273441,chama precaucao medicamento barato caso venha ...,chama precaucao medicamento barato caso venha ...,27236,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9052732...,162513,968,318,88,509,20723,True


In [41]:
print('número de posts do conjunto de teste (30%): {0}'.format(str(len(df_test))))
df_test.head(5)

número de posts do conjunto de teste (30%): 1094


,created_time,id,m_m_p_replaced_to_abbreviation,m_m_p_replaced_to_empty,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
2552,2016-07-07T18:10:47+0000,211857482296579_654534418028881,obrigado fortaleza,obrigado fortaleza,3882,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/13...,23172,48,179,0,6,682,True
2553,2016-07-07T17:27:29+0000,211857482296579_654521978030125,assistam vejam link abaixo razao dessa manifes...,assistam vejam link abaixo razao dessa manifes...,951,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/13...,10537,14,58,0,4,248,True
2554,2016-07-06T18:57:53+0000,211857482296579_654105648071758,ratinho general candidato presidencia republic...,ratinho general candidato presidencia republic...,48455,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/13...,70643,521,202,23,36,2279,True
2555,2016-07-05T21:55:08+0000,211857482296579_653748394774150,jb fortaleza quintafeira jul,fortaleza quintafeira jul,1500,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/13...,13768,39,12,0,3,305,True
2556,2016-07-05T19:28:05+0000,211857482296579_653691854779804,coronel ustra terrorista marighella dep jb,coronel ustra terrorista marighella dep,10286,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/13...,17289,9,222,13,50,372,True


In [44]:
#============================================================================================
# realizando o tf idf no df['pre_processed_message'] de treinamento
#============================================================================================

# tf idf é uma medida estatística que tem o intuito de indicar a importância de uma palavra 
# de um documento em relação a uma coleção de documentos ou em um corpus linguístico.

# instanciando o objeto para fazer o tf-idf
tfidf_vectorizer = TfidfVectorizer(analyzer='word')

# converte os documentos em matrizes
tfidf_matrix = tfidf_vectorizer.fit_transform(df_training['pre_processed_message'])

# obtendo as palavras (tokens)
tfidf_tokens = tfidf_vectorizer.get_feature_names()

# gerando o tf-idf
# colunas são as palavras processadas
# linhas são os documentos (post 1, post 2,post 3,post 4)
df_tfidfvect = pd.DataFrame(data = tfidf_matrix.toarray(),columns = tfidf_tokens)


print("\nTF-IDF\n")
df_tfidfvect.head(10)

KeyError: 'pre_processed_message'

In [43]:
# Aplicando uma máscara para identificar o tf-idf de certas palavras

filter_mask  = df_tfidfvect['abaixo'] != 0.0
df_tfidf_token_mask = df_tfidfvect[filter_mask]

print(df_tfidf_token_mask)

NameError: name 'df_tfidfvect' is not defined

In [72]:
#============================================================================================
# Implementacao do lda
#============================================================================================

In [73]:
cv = CountVectorizer()
df = cv.fit_transform(df_training['pre_processed_message'])

In [74]:
model_lda = LatentDirichletAllocation(n_components = 8,random_state = 0)
model_lda.fit(df)

LatentDirichletAllocation(n_components=8, random_state=0)

In [75]:
for index, topic in enumerate(model_lda.components_):
    print(f'Top 10 words for #{index}')
    main_topics = [cv.get_feature_names()[i] for i in topic.argsort()[-10:]]
    print(main_topics)
    message = ' '
    message = message.join(main_topics)
    print(message)
    print('\n')
    

Top 10 words for #0
['inscrevase', 'sao', 'nao', 'paulo', 'canal', 'brasil', 'jair', 'bolsonaro', 'link', 'youtube']
inscrevase sao nao paulo canal brasil jair bolsonaro link youtube


Top 10 words for #1
['ministro', 'milhoes', 'ano', 'estado', 'nao', 'presidente', 'governo', 'brasil', 'bolsonaro', 'ministerio']
ministro milhoes ano estado nao presidente governo brasil bolsonaro ministerio


Top 10 words for #2
['dia', 'sao', 'bilhoes', 'tudo', 'nao', 'bolsonaro', 'deus', 'todos', 'acima', 'brasil']
dia sao bilhoes tudo nao bolsonaro deus todos acima brasil


Top 10 words for #3
['pt', 'militar', 'ministerio', 'sao', 'saude', 'boa', 'noite', 'brasil', 'nao', 'todos']
pt militar ministerio sao saude boa noite brasil nao todos


Top 10 words for #4
['so', 'jair', 'corrupcao', 'governo', 'eduardo', 'povo', 'via', 'nao', 'brasil', 'bolsonaro']
so jair corrupcao governo eduardo povo via nao brasil bolsonaro


Top 10 words for #5
['aeroporto', 'brasil', 'grande', 'nao', 'todos', 'sao', 'bol